# Airline Portfolio Optimization Project

---
Rodrigo Soriano & Philip Bachas-Daunert

**Option 1: Pick your 20 favorite stocks.**

In [39]:
# @title Importing Modules { run: "auto" }

!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp # import module (or package) with the name gurobipy and abbreviate it as gp
from gurobipy import Model, GRB # from gurobipy import Model & GRB solver
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import yfinance as yf

#Abstract


---


In present-day Miami, Rodrigo and Philip work in an investment company that caters to eccentric upper-class clients. One day, Tom Cruise enters the office unannounced and requests the following from the young, upstart investors:

“My son, Connor Antony, has had trouble finding a direction in life. He is 28 years old and has recently told me that he aspires to be a day trader. As my friend David Miscavige says, ‘one of our important maxims is that a being is as valuable as one is able to help others.’ I want to help my son but I have some constraints, as the budget for ['Mission: Impossible - Dead Reckoning Part Two'](https://www.imdb.com/title/tt9603208/) has ballooned quite a bit recently since we keep trying to up the stakes of the action.”

## Constraints


---


### (1) A NASDAQ/NYSE Portfolio of 20 Airline companies with financial data dating back to July 2019.

“I have had a passion for aviation ever since shooting the movie ['Top Gun'](https://www.imdb.com/title/tt0092099/) and it has rubbed off on Connor Antony. I figured since we both are passionate about this field I can help him daily to understand industry trends.”

$A = \{a_1, a_2, \ldots, a_{20}\}$

### (2) Initial Investment is $1,000,000.

“I know it is a meager sum to start a portfolio, but being a producer has me pinching pennies in other areas for the sake of ['Mission: Impossible - Dead Reckoning Part Two'](https://www.imdb.com/title/tt9603208/).”

$I = 1,000,000$

### (3) Upper Limit of Risk Tolerance of $\sigma = 30\%$.

“Connor Antony can take a lot of risk in this venture as I may need some extra funds later to fund ['Mission: Impossible - Dead Reckoning Part Two'](https://www.imdb.com/title/tt9603208/). As I learned from my time on the set of ['Rain Man'](https://www.imdb.com/title/tt0095953/), sometimes you have to take risks to make a lot of money.”

$ \sigma(R) \leq 0.30$

### (4) Constraint on Individual Weights

This is a follow-up addition to added by Rodrigo and Philip.

$w_i \leq 0.1337 \quad \forall i \in \{1, 2, \ldots, n\}$

# The 'Problem' Data


---

We will be solving this portfolio optimization problem by using the Markowitz framework.

\begin{align*}
\text{Maximize} \quad & E(R) \\
\text{subject to} \quad & \sum_{i=1}^{20} w_i \cdot r_i = E(R) \\
& \sum_{i=1}^{20} w_i = 1 \\
& w_i \cdot I = X_i, \quad i=1,\ldots,20 \\
& \sigma(R) \leq 0.30 \\
&w_i \leq 0.1337, \quad i=1,\ldots,20
\end{align*}

Here:
- $E(R)$ is the expected return of the portfolio.
- $r_i$ is the expected return of asset $i$.
- $w_i$ is the proportion of total investment in asset $i$.
- $X_i$ is the amount of money invested in asset $i$.
- $\sigma(R)$ is the risk of the portfolio.
- $I = 1,000,000$

The added complication of Tom Cruise's request is how volatile the airline industry has been since the COVID-19 Pandemic started in late 2019. Rodrigo and Philip are not quite sure if this part of the challenge for Connor Antony or if Tom Cruise just really loves airplanes and the airline industry. Read this International Air Transport Association funded article from December 2022 for a holistic understanding of this industry's volatility: [Understanding the pandemic’s impact on the aviation value chain](https://www.iata.org/en/iata-repository/publications/economic-reports/understanding-the-pandemics-impact-on-the-aviation-value-chain/).

## Class Notes on Portfolio Selection Optimization for Reference
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

### Model Formulation
#### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $\delta$: n-element vector measuring the change in price for each stock
- $\sigma$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


#### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the porfolio to invest in each stock

#### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

#### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):

\begin{equation}
e \cdot x = 1
\end{equation}


Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:

\begin{equation}
\delta \cdot x = r
\end{equation}

# Part (A)

## Collect data about the stocks' returns (based on closing price/month) for, at the minimum, the last 3 months. You can find such data on the internet, for example at [Yahoo! Finance](https://finance.yahoo.com/lookup).



---
Here is a list of the 20 airline stocks we are going to analyze. The period is from 04-2021 to 10-2023. Please note that based upon the Investor Relations Annual Review Documents some airlines have FYs that do not end in December.:

* AAL
* DAL
* UAL
* LUV
* RYAAY
* JBLU
* SAVE
* VLRS
* ULCC
* GOL
* AZUL
* CPA
* HA
* MESA
* ALK
* SNCY
* ALGT
* ATSG
* UP
* SKYW





In [40]:
# 20 Airlines Stock
stocks = ['AAL', 'ALGT', 'ALK', 'ATSG', 'AZUL',
          'CPA', 'DAL', 'GOL', 'HA', 'JBLU',
          'LUV', 'MESA', 'RYAAY', 'SAVE', 'SKYW',
          'SNCY', 'UAL', 'ULCC', 'UP', 'VLRS']

# Loading in the Dataset here.
data = yf.download(stocks , period = '30mo')

[*********************100%%**********************]  20 of 20 completed


In [41]:
# Showing the data loaded properly
print(data)

           Adj Close                                                        \
                 AAL     ALGT      ALK     ATSG     AZUL      CPA      DAL   
Date                                                                         
2021-04-05  2.42e+01 2.49e+02 7.11e+01 3.06e+01 2.10e+01 7.86e+01 5.01e+01   
2021-04-06  2.41e+01 2.50e+02 7.37e+01 3.07e+01 2.11e+01 8.03e+01 5.15e+01   
2021-04-07  2.39e+01 2.49e+02 7.24e+01 2.92e+01 2.11e+01 7.83e+01 5.01e+01   
2021-04-08  2.36e+01 2.50e+02 7.27e+01 2.92e+01 2.16e+01 7.92e+01 4.97e+01   
2021-04-09  2.35e+01 2.51e+02 7.26e+01 2.94e+01 2.08e+01 7.87e+01 4.92e+01   
...              ...      ...      ...      ...      ...      ...      ...   
2023-09-29  1.28e+01 7.69e+01 3.71e+01 2.09e+01 8.59e+00 8.91e+01 3.70e+01   
2023-10-02  1.28e+01 7.53e+01 3.68e+01 2.08e+01 8.05e+00 8.64e+01 3.65e+01   
2023-10-03  1.23e+01 7.40e+01 3.58e+01 2.07e+01 7.50e+00 8.34e+01 3.52e+01   
2023-10-04  1.27e+01 7.58e+01 3.61e+01 2.10e+01 7.68e+00 8.51e+0

In [42]:
# Check for NaN values
has_nans = data.isna().any().any()

print(f"Does the DataFrame have NaN values? {'Yes' if has_nans else 'No'}")

Does the DataFrame have NaN values? No


# Part (B)
## Compute the average return and the covariance matrix.

In [43]:
# Code from Professor

closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
delta = np.mean(reldiff, axis=1)            # mean price change
sigma = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation


# Part (C)
## Plot the efficient frontier of your optimal portfolio of minimum variance.

In [44]:
# Create a model
m = gp.Model('portfolio')

# Add variable for the stocks
x = m.addMVar(len(stocks))

# Add a risk constraint
m.addConstr(x @ sigma @ x <= 0.30**2, 'risk')

# Add weight constraints for each stock

upper_bound = 0.1337  # Upper weight limit per stock

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = x @ sigma @ x
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with a constraint
m.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x60c799ac
Model has 210 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e-04, 5e-03]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-04, 1e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [9e-02, 9e-02]
Presolve time: 0.01s
Presolved: 43 rows, 63 columns, 483 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.600e+02
 Factor NZ  : 9.460e+02
 Factor Ops : 2.743e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dua

In [45]:
minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = delta @ x.X
pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
             index=stocks + ['Volatility', 'Expected Return'],
             columns=['Minimum Risk Portfolio'])

,Minimum Risk Portfolio
AAL,6.84e-07
ALGT,6.41e-06
ALK,2.94e-05
ATSG,3.09e-01
AZUL,2.57e-05
CPA,1.48e-01
DAL,1.94e-06
GOL,2.59e-06
HA,6.10e-07
JBLU,1.61e-06


In [ ]:
# Create an expression representing the expected return for the portfolio
portfolio_return = delta @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x197e2609
Model has 210 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  QMatrix range    [2e-04, 5e-03]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-04, 1e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]
  QRHS range       [9e-02, 9e-02]
Presolve time: 0.03s
Presolved: 44 rows, 63 columns, 503 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.010e+02
 Factor NZ  : 9.900e+02
 Factor Ops : 2.937e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dua

In [ ]:
# Efficient Frontier Graph

fig, ax = plt.subplots(figsize=(10,8))

# Plot volatility versus expected return for individual stocks
ax.scatter(x=std, y=delta,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (std[i], delta[i]))

# Plot volatility versus expected return for minimum risk portfolio
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# Plot efficient frontier
ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# Format and display the final plot
ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()

#Part (D)
## Set your target return to be equal to the average return across all stocks. What is your optimal portfolio?

In [ ]:
# Assume 'stocks' and 'sigma' are defined above the provided code.

# Create a model
m = gp.Model('portfolio')

# Add variable for the stocks
x = m.addMVar(len(stocks))

# Constraint: Overall risk of the portfolio should not exceed 0.30
m.addConstr(x @ sigma @ x <= 0.30**2, 'risk')

# Constraint: Weight of each individual stock should not exceed 13.37% (0.1337)
upper_bound = 0.1337
for i in range(len(stocks)):
    m.addConstr(x[i] <= upper_bound, f'upper_weight_{stocks[i]}')

# Objective: minimize the portfolio risk, modeled using covariance matrix sigma
m.setObjective(x @ sigma @ x, GRB.MINIMIZE)

# Constraint: Sum of weights should be equal to 1 (i.e., full investment)
m.addConstr(x.sum() == 1, 'budget')

# Optimize model to find the minimum risk portfolio
m.optimize()

# Ensure that an optimal solution is found
if m.status == GRB.Status.OPTIMAL:
    print('Optimal solution found!')
else:
    print('No optimal solution found. Status code:', m.status)

# Plotting optimal portfolio weights
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(stocks, optimal_weights)
ax.set_xlabel('Stock')
ax.set_ylabel('Optimal Weight')
ax.set_title('Optimal Portfolio Weights')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plotting USD amount per stock
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(stocks, amounts_invested)
ax.set_xlabel('Stock')
ax.set_ylabel('USD Amount Invested')
ax.set_title('USD Amount Invested in Each Stock in the Optimal Portfolio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
pd.set_option('display.float_format', '{:.2e}'.format)

# Create DataFrame for Optimal Portfolio Weights
df_weights = pd.DataFrame({
    'Stock': stocks,
    'Optimal Weight': optimal_weights
})
print("\nOptimal Portfolio Weights:")
print(df_weights)

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

# Create DataFrame for USD Amount Invested in Each Stock
df_invested = pd.DataFrame({
    'Stock': stocks,
    'USD Amount Invested': amounts_invested
})
df_invested['USD Amount Invested'] = df_invested['USD Amount Invested'].round(2)
print("\nUSD Amount Invested in Each Stock:")
print(df_invested)


# Conclusion


---

Six months later, Tom Cruise storms into Rodrigo and Philip’s office, a fierce glare on his face.

"I know ['The Color of Money'](https://www.imdb.com/title/tt0090863/) is green, but I see red all over Connor Antony’s airline portfolio! What is going on?!” scowled Tom Cruise.

Rodrigo and Philip quickly presented Tom Cruise with a detailed Jupyter Notebook explanation of their portfolio optimization via the Markowitz framework. The pair demonstrated that their efficient frontier figure had skewed data, resulting in a poor efficient frontier indication—not ideal for a profitable and diverse portfolio. Even the inclusion of a constraint for the upper bound on the 20 stocks’ Individual Weights did little to enhance the diversity and profitability of the portfolio.


* [AAL](https://finance.yahoo.com/quote/AAL) | Weight: 1.95e-08 | Initial Investment: \$0.02
* [ALGT](https://finance.yahoo.com/quote/ALGT) | Weight: 1.76e-04 | Initial Investment: \$176.37
* [ALK](https://finance.yahoo.com/quote/ALK) | Weight: 1.34e-01 | Initial Investment: \$133,699.96
* [ATSG](https://finance.yahoo.com/quote/ATSG) | Weight: 1.34e-01 | Initial Investment: \$133,699.99
* [AZUL](https://finance.yahoo.com/quote/AZUL) | Weight: 8.57e-07 | Initial Investment: \$0.86
* [CPA](https://finance.yahoo.com/quote/CPA) | Weight: 1.34e-01 | Initial Investment: \$133,699.98
* [DAL](https://finance.yahoo.com/quote/DAL) | Weight: 7.78e-02 | Initial Investment: \$77,775.10
* [GOL](https://finance.yahoo.com/quote/GOL) | Weight: 5.20e-08 | Initial Investment: \$0.05
* [HA](https://finance.yahoo.com/quote/HA) | Weight: 1.30e-08 | Initial Investment: \$0.01
* [JBLU](https://finance.yahoo.com/quote/JBLU) | Weight: 6.19e-08 | Initial Investment: \$0.06
* [LUV](https://finance.yahoo.com/quote/LUV) | Weight: 1.34e-01 | Initial Investment: \$133,699.99
* [MESA](https://finance.yahoo.com/quote/MESA) | Weight: 3.72e-08 | Initial Investment: \$0.04
* [RYAAY](https://finance.yahoo.com/quote/RYAAY) | Weight: 1.34e-01 | Initial Investment: \$133,699.99
* [SAVE](https://finance.yahoo.com/quote/SAVE) | Weight: 1.34e-01 | Initial Investment: \$133,699.94
* [SKYW](https://finance.yahoo.com/quote/SKYW) | Weight: 2.71e-03 | Initial Investment: \$2,710.52
* [SNCY](https://finance.yahoo.com/quote/SNCY) | Weight: 7.35e-02 | Initial Investment: \$73,486.24
* [UAL](https://finance.yahoo.com/quote/UAL) | Weight: 2.18e-08 | Initial Investment: \$0.02
* [ULCC](https://finance.yahoo.com/quote/ULCC) | Weight: 3.63e-08 | Initial Investment: \$0.04
* [UP](https://finance.yahoo.com/quote/UP) | Weight: 6.15e-03 | Initial Investment: \$6,145.84
* [VLRS](https://finance.yahoo.com/quote/VLRS) | Weight: 3.75e-02 | Initial Investment: \$37,504.97

Rodrigo and Philip confidently asserted to Tom Cruise that this was a pivotal lesson for Connor Antony: portfolios need to have diversification across multiple industries and company sizes. Tom Cruise nodded, remarking, “Well, at least this wasn’t a costly lesson, as I have recently secured extra funding for ['Mission: Impossible - Dead Reckoning Part Two'](https://www.imdb.com/title/tt9603208/) from the studio!” As Tom Cruise exited the office, reassured of this financial lesson for Connor Antony, he overheard Rodrigo and Philip muse, “Day trading is a ['Risky Business'](https://www.imdb.com/title/tt0086200/) without diversification in your portfolio.”